## Interaktivni Phi 3 Mini 4K Instruct Chatbot z Whisper

### Uvod:
Interaktivni Phi 3 Mini 4K Instruct Chatbot je orodje, ki uporabnikom omogoča interakcijo z Microsoft Phi 3 Mini 4K Instruct demo prek besedilnega ali zvočnega vnosa. Chatbot se lahko uporablja za različne naloge, kot so prevajanje, posodobitve vremena in zbiranje splošnih informacij.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Ustvarite svoj Huggingface dostopni žeton

Ustvarite nov žeton  
Dajte mu novo ime  
Izberite dovoljenja za pisanje  
Kopirajte žeton in ga shranite na varno mesto


Naslednja Python koda izvaja dve glavni nalogi: uvoz modula `os` in nastavitev okoljske spremenljivke.

1. Uvoz modula `os`:
   - Modul `os` v Pythonu omogoča interakcijo z operacijskim sistemom. Omogoča izvajanje različnih nalog, povezanih z operacijskim sistemom, kot so dostop do okoljskih spremenljivk, delo z datotekami in imeniki itd.
   - V tej kodi je modul `os` uvožen z uporabo ukaza `import`. Ta ukaz omogoča uporabo funkcionalnosti modula `os` v trenutnem Python skriptu.

2. Nastavitev okoljske spremenljivke:
   - Okoljska spremenljivka je vrednost, do katere lahko dostopajo programi, ki se izvajajo na operacijskem sistemu. Gre za način shranjevanja nastavitev konfiguracije ali drugih informacij, ki jih lahko uporabljajo različni programi.
   - V tej kodi se nova okoljska spremenljivka nastavi z uporabo slovarja `os.environ`. Ključ slovarja je `'HF_TOKEN'`, vrednost pa je dodeljena iz spremenljivke `HUGGINGFACE_TOKEN`.
   - Spremenljivka `HUGGINGFACE_TOKEN` je definirana tik nad tem delom kode in ji je dodeljena nizovna vrednost `"hf_**************"` z uporabo sintakse `#@param`. Ta sintaksa se pogosto uporablja v Jupyter zvezkih za omogočanje vnosa uporabnika in konfiguracijo parametrov neposredno v vmesniku zvezka.
   - Z nastavitvijo okoljske spremenljivke `'HF_TOKEN'` lahko do nje dostopajo drugi deli programa ali drugi programi, ki se izvajajo na istem operacijskem sistemu.

Na splošno ta koda uvozi modul `os` in nastavi okoljsko spremenljivko z imenom `'HF_TOKEN'` z vrednostjo, ki je podana v spremenljivki `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Ta del kode definira funkcijo z imenom clear_output, ki se uporablja za brisanje izpisa trenutne celice v Jupyter Notebooku ali IPythonu. Poglejmo si podrobnosti kode in razumimo njeno funkcionalnost:

Funkcija clear_output sprejme en parameter, imenovan wait, ki je logična vrednost (boolean). Privzeto je wait nastavljen na False. Ta parameter določa, ali naj funkcija počaka, da je na voljo nov izpis za zamenjavo obstoječega izpisa, preden ga izbriše.

Sama funkcija se uporablja za brisanje izpisa trenutne celice. V Jupyter Notebooku ali IPythonu, ko celica ustvari izpis, kot je natisnjeno besedilo ali grafični prikazi, se ta izpis prikaže pod celico. Funkcija clear_output omogoča, da ta izpis izbrišete.

Implementacija funkcije ni podana v tem delčku kode, kar je označeno z elipso (...). Elipsa predstavlja mesto za dejansko kodo, ki izvaja brisanje izpisa. Implementacija funkcije lahko vključuje interakcijo z API-jem Jupyter Notebooka ali IPythona za odstranitev obstoječega izpisa iz celice.

Na splošno ta funkcija ponuja priročen način za brisanje izpisa trenutne celice v Jupyter Notebooku ali IPythonu, kar olajša upravljanje in posodabljanje prikazanega izpisa med interaktivnimi programskimi sejami.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Izvedite pretvorbo besedila v govor (TTS) z uporabo storitve Edge TTS. Poglejmo si ustrezne implementacije funkcij eno za drugo:

1. `calculate_rate_string(input_value)`: Ta funkcija sprejme vhodno vrednost in izračuna niz hitrosti za glas TTS. Vhodna vrednost predstavlja želeno hitrost govora, kjer vrednost 1 predstavlja normalno hitrost. Funkcija izračuna niz hitrosti tako, da od vhodne vrednosti odšteje 1, jo pomnoži s 100 in nato določi predznak glede na to, ali je vhodna vrednost večja ali enaka 1. Funkcija vrne niz hitrosti v formatu "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ta funkcija sprejme vhodno besedilo in jezik kot parametra. Besedilo razdeli na dele glede na jezikovno specifična pravila. V tej implementaciji, če je jezik "English", funkcija razdeli besedilo na vsaki piki (".") in odstrani morebitne začetne ali končne presledke. Nato vsakemu delu doda piko in vrne filtriran seznam delov.

3. `tts_file_name(text)`: Ta funkcija ustvari ime datoteke za zvočno datoteko TTS na podlagi vhodnega besedila. Izvede več transformacij na besedilu: odstrani končno piko (če je prisotna), pretvori besedilo v male črke, odstrani začetne in končne presledke ter zamenja presledke z podčrtaji. Nato besedilo skrajša na največ 25 znakov (če je daljše) ali uporabi celotno besedilo, če je prazno. Na koncu ustvari naključni niz z uporabo modula [`uuid`] in ga združi s skrajšanim besedilom, da ustvari ime datoteke v formatu "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ta funkcija združi več zvočnih datotek v eno zvočno datoteko. Sprejme seznam poti zvočnih datotek in izhodno pot kot parametra. Funkcija inicializira prazen objekt `AudioSegment`, imenovan [`merged_audio`]. Nato iterira skozi vsako pot zvočne datoteke, naloži zvočno datoteko z metodo `AudioSegment.from_file()` iz knjižnice `pydub` in doda trenutno zvočno datoteko objektu [`merged_audio`]. Na koncu izvozi združeno zvočno datoteko na določeno izhodno pot v formatu MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ta funkcija izvede operacijo TTS z uporabo storitve Edge TTS. Sprejme seznam delov besedila, hitrost govora, ime glasu in pot shranjevanja kot parametre. Če je število delov večje od 1, funkcija ustvari imenik za shranjevanje posameznih zvočnih datotek delov. Nato iterira skozi vsak del, sestavi ukaz Edge TTS z uporabo funkcije `calculate_rate_string()`, imena glasu in besedila dela ter izvede ukaz z uporabo funkcije `os.system()`. Če je izvedba ukaza uspešna, doda pot ustvarjene zvočne datoteke v seznam. Po obdelavi vseh delov združi posamezne zvočne datoteke z uporabo funkcije `merge_audio_files()` in shrani združeno zvočno datoteko na določeno pot shranjevanja. Če je samo en del, neposredno ustvari ukaz Edge TTS in shrani zvočno datoteko na pot shranjevanja. Na koncu vrne pot shranjene zvočne datoteke.

6. `random_audio_name_generate()`: Ta funkcija ustvari naključno ime zvočne datoteke z uporabo modula [`uuid`]. Ustvari naključni UUID, ga pretvori v niz, vzame prvih 8 znakov, doda pripono ".mp3" in vrne naključno ime zvočne datoteke.

7. `talk(input_text)`: Ta funkcija je glavni vstopni točki za izvedbo operacije TTS. Sprejme vhodno besedilo kot parameter. Najprej preveri dolžino vhodnega besedila, da določi, ali gre za dolg stavek (večji ali enak 600 znakov). Na podlagi dolžine in vrednosti spremenljivke `translate_text_flag` določi jezik in ustvari seznam delov besedila z uporabo funkcije `make_chunks()`. Nato ustvari pot shranjevanja za zvočno datoteko z uporabo funkcije `random_audio_name_generate()`. Na koncu pokliče funkcijo `edge_free_tts()` za izvedbo operacije TTS in vrne pot shranjene zvočne datoteke.

Skupno te funkcije delujejo skupaj, da razdelijo vhodno besedilo na dele, ustvarijo ime datoteke za zvočno datoteko, izvedejo operacijo TTS z uporabo storitve Edge TTS in združijo posamezne zvočne datoteke v eno zvočno datoteko.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementacija dveh funkcij: convert_to_text in run_text_prompt, ter deklaracija dveh razredov: str in Audio.

Funkcija convert_to_text sprejme parameter audio_path in prepiše zvok v besedilo z uporabo modela whisper_model. Funkcija najprej preveri, ali je zastavica gpu nastavljena na True. Če je, se whisper_model uporabi z določenimi parametri, kot so word_timestamps=True, fp16=True, language='English' in task='translate'. Če je gpu nastavljena na False, se whisper_model uporabi z nastavitvijo fp16=False. Nastali prepis se nato shrani v datoteko z imenom 'scan.txt' in vrne kot besedilo.

Funkcija run_text_prompt sprejme sporočilo in zgodovino klepeta kot vhodne parametre. Uporabi funkcijo phi_demo za generiranje odgovora klepetalnega robota na podlagi vhodnega sporočila. Generirani odgovor se nato posreduje funkciji talk, ki pretvori odgovor v zvočno datoteko in vrne pot do datoteke. Razred Audio se uporablja za prikaz in predvajanje zvočne datoteke. Zvok se prikaže z uporabo funkcije display iz modula IPython.display, objekt Audio pa se ustvari s parametrom autoplay=True, tako da se zvok začne predvajati samodejno. Zgodovina klepeta se posodobi z vhodnim sporočilom in generiranim odgovorom, nato pa se vrnejo prazna vrstica in posodobljena zgodovina klepeta.

Razred str je vgrajen razred v Pythonu, ki predstavlja zaporedje znakov. Ponuja različne metode za manipulacijo in delo z nizi, kot so capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill in še več. Te metode omogočajo izvajanje operacij, kot so iskanje, zamenjava, formatiranje in manipulacija nizov.

Razred Audio je prilagojen razred, ki predstavlja zvočni objekt. Uporablja se za ustvarjanje zvočnega predvajalnika v okolju Jupyter Notebook. Razred sprejme različne parametre, kot so data, filename, url, embed, rate, autoplay in normalize. Parameter data je lahko numpy array, seznam vzorcev, niz, ki predstavlja ime datoteke ali URL, ali surovi PCM podatki. Parameter filename se uporablja za določanje lokalne datoteke, iz katere se naložijo zvočni podatki, parameter url pa za določanje URL-ja, s katerega se prenesejo zvočni podatki. Parameter embed določa, ali naj bodo zvočni podatki vdelani z uporabo URI podatkov ali referencirani iz izvirnega vira. Parameter rate določa vzorčno frekvenco zvočnih podatkov. Parameter autoplay določa, ali naj se zvok začne predvajati samodejno. Parameter normalize določa, ali naj se zvočni podatki normalizirajo (prilagodijo) na največji možni razpon. Razred Audio ponuja tudi metode, kot je reload za ponovno nalaganje zvočnih podatkov iz datoteke ali URL-ja, ter atribute, kot so src_attr, autoplay_attr in element_id_attr za pridobivanje ustreznih atributov za zvočni element v HTML-ju.

Na splošno se te funkcije in razredi uporabljajo za prepis zvoka v besedilo, generiranje zvočnih odgovorov klepetalnega robota ter prikaz in predvajanje zvoka v okolju Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
